In [11]:
#import necessary libraries
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import scipy.ndimage as nd
from tensorflow import keras
%matplotlib inline
import pylab as pl
from IPython import display


import sys
sys.path.append("..") # Adds higher directory to python modules path.

import branchingdnn as branching
from branchingdnn import core
from branchingdnn.utils import * 
from branchingdnn.profiler import model_profiler as profiler
from branchingdnn.dataset import prepare
from branchingdnn.branches import branch

# from tensorflow.examples.tutorials.mnist import input_data

In [12]:
# Download MNIST dataset
# mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# print(y_train)
K= 10 # number of classes

In [15]:
train_images = train_images.reshape(60000, 784).astype("float32") / 255
test_images = test_images.reshape(10000, 784).astype("float32") / 255


train_labels = tf.keras.utils.to_categorical(train_labels,10)
test_labels = tf.keras.utils.to_categorical(test_labels,10)

print(len(train_labels))
print(len(test_labels))
# print(train_labels[0])
# print((test_labels[0]))
# plt.imshow(train_images[1].reshape(228,228)) 
# plt.show()

60000
10000


In [31]:
validation_size = 12000
shuffle_size = 20000
batch_size=128
validation_images, validation_labels = train_images[:validation_size], train_labels[:validation_size] #get the first 5k training samples as validation set
train_images, train_labels = train_images[validation_size:], train_labels[validation_size:] # now remove the validation set from the training set.
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))


def augment_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    # image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    # image = tf.image.resize(image, (28,28))
    return image, label
train_ds_size = len(list(train_ds))
test_ds_size = len(list(test_ds))
validation_ds_size = len(list(validation_ds))
train_ds = (train_ds.map(augment_images))
validation_ds = (validation_ds.map(augment_images))
test_ds = (test_ds.map(augment_images))

target = tf.data.Dataset.from_tensor_slices((train_labels))
train_ds = tf.data.Dataset.zip((train_ds,target))

v_target = tf.data.Dataset.from_tensor_slices((validation_labels))
validation_ds = tf.data.Dataset.zip((validation_ds,v_target))

t_target = tf.data.Dataset.from_tensor_slices((test_labels))
test_ds = tf.data.Dataset.zip((test_ds,t_target))


print("trainSize {}".format(train_ds_size))
print("testSize {}".format(test_ds_size))
train_ds = (train_ds
                
                .shuffle(buffer_size=tf.cast(shuffle_size,'int64'))
                .batch(batch_size=batch_size, drop_remainder=True))

test_ds = (test_ds
               
                #   .shuffle(buffer_size=train_ds_size)
                .batch(batch_size=1, drop_remainder=True))

validation_ds = (validation_ds
               
                #   .shuffle(buffer_size=validation_ds_size)
                .batch(batch_size=batch_size, drop_remainder=True))

trainSize 24000
testSize 10000


In [32]:
# This function to generate evidence is used for the first example
def relu_evidence(logits):
    return tf.nn.relu(logits)

# This one usually works better and used for the second and third examples
# For general settings and different datasets, you may try this one first
def exp_evidence(logits): 
    return tf.exp(tf.clip_by_value(logits,-10,10))

# This one is another alternative and 
# usually behaves better than the relu_evidence 
def softplus_evidence(logits):
    return tf.nn.softplus(logits)
    
def KL(alpha):
    # print("K:",K)
    beta=tf.constant(np.ones((1,K)),dtype=tf.float32)
    S_alpha = tf.reduce_sum(alpha,axis=1,keepdims=True)
    S_beta = tf.reduce_sum(beta,axis=1,keepdims=True)
    lnB = tf.compat.v1.lgamma(S_alpha) - tf.reduce_sum(tf.compat.v1.lgamma(alpha),axis=1,keepdims=True)
    lnB_uni = tf.reduce_sum(tf.compat.v1.lgamma(beta),axis=1,keepdims=True) - tf.compat.v1.lgamma(S_beta)
    
    dg0 = tf.compat.v1.digamma(S_alpha)
    dg1 = tf.compat.v1.digamma(alpha)
    
    kl = tf.reduce_sum((alpha - beta)*(dg1-dg0),axis=1,keepdims=True) + lnB + lnB_uni
    # print("kl", kl)
    return kl

def mse_loss(p, alpha, global_step, annealing_step): 
    S = tf.reduce_sum(alpha, axis=1, keepdims=True) 
    E = alpha - 1
    m = alpha / S
    
    A = tf.reduce_sum((p-m)**2, axis=1, keepdims=True) 
    B = tf.reduce_sum(alpha*(S-alpha)/(S*S*(S+1)), axis=1, keepdims=True) 

    annealing_coef = tf.minimum(1.0,tf.cast(global_step/annealing_step,tf.float32))
    
    alp = E*(1-p) + 1 
    # print("alp", alp)

    C =  annealing_coef * KL(alp)
    return (A + B) + C

def loss_function():
    #create a wrapper function that returns a function
    kl = tf.keras.losses.KLDivergence()
    temperature = 1
    Classes = 10
    crossE = tf.keras.losses.CategoricalCrossentropy()
    mse =mse_loss
    global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
    annealing_step = 320
    def EMSE_Loss(y_true, y_pred):
#         softmax = tf.nn.softmax(y_pred)
        evidence = exp_evidence(y_pred)
        alpha  = evidence + 1
        
        u = Classes / tf.reduce_sum(alpha, axis=1) #uncertainty
        
        prob = alpha/tf.reduce_sum(alpha, 1,keepdims=True) 
        
        loss = tf.reduce_mean(mse(y_true, alpha,global_step,annealing_step))
#         l2_loss = (tf.nn.l2_loss(W3)+tf.nn.l2_loss(W4)) * lmb
#         l2_loss = tf.reduce_sum(tf.square(inputs - teaching_features))
        
        final_loss = loss #+ l2_loss
        
#         kl_loss = kl( tf.nn.softmax(softmax / self.temperature, axis = 1 ),
        
        return final_loss
        
    def EMSE_Loss_evidence(y_true, y_pred):
#         softmax = tf.nn.softmax(y_pred)
        evidence = exp_evidence(y_pred)
        alpha  = evidence + 1
        
        u = Classes / tf.reduce_sum(alpha, axis=1) #uncertainty
        
        prob = alpha/tf.reduce_sum(alpha, 1,keepdims=True) 
        
        loss = tf.reduce_mean(mse(y_true, alpha,global_step,annealing_step))
#         l2_loss = (tf.nn.l2_loss(W3)+tf.nn.l2_loss(W4)) * lmb
#         l2_loss = tf.reduce_sum(tf.square(inputs - teaching_features))
        
        final_loss = loss #+ l2_loss
        
#         kl_loss = kl( tf.nn.softmax(softmax / self.temperature, axis = 1 ),
        
        return evidence, final_loss
    return  EMSE_Loss



In [33]:
class EvidenceEndpoint(keras.layers.Layer):
        def __init__(self, num_outputs, name=None, **kwargs):
            super(EvidenceEndpoint, self).__init__(name=name)
            self.num_outputs = num_outputs
            self.loss_coefficient = 1
            self.feature_loss_coefficient = 1
            self.kl = tf.keras.losses.KLDivergence()
            self.loss_fn = loss_function()
            self.evidence = exp_evidence
            self.temperature = 10
            self.alpha = .1
            self.lmb = 0.005
        def build(self, input_shape):
            self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.num_outputs])
        
        def get_config(self):
            config = super().get_config().copy()
            config.update({
                'num_outputs': self.num_outputs,
                'name': self.name
            })
            return config

        def call(self, inputs, labels):
            outputs = tf.matmul(inputs,self.kernel)
            # Compute the training-time loss value and add it
            # to the layer using `self.add_loss()`.
            #loss functions are (True, Prediction)
            # softmax = tf.nn.softmax(inputs)
            # print("inputs",input)
            #loss 1. normal loss, predictions vs labels
            normal_loss = self.loss_fn(labels, outputs)
            # evidence = self.evidence(outputs)
            evidence = exp_evidence(outputs)

            l2_loss = tf.nn.l2_loss(self.weights) * self.lmb
            # print(evidence.shape)
            # print(normal_loss)
            total_loss =tf.reduce_mean(normal_loss) + l2_loss
            # print(total_loss)
            total_evidence = tf.reduce_sum(evidence,1, keepdims=True) 
            # print(total_evidence)
            pred = tf.argmax(outputs, 1)
            truth = tf.argmax(labels, 1)
            match = tf.reshape(tf.cast(tf.equal(pred, truth), tf.float32),(-1,1))
            # print("match",match)
            mean_avg = tf.reduce_mean(total_evidence)
            # print("mean_Avg")
            
            mean_succ = tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)*match) / tf.reduce_sum(match+1e-20)
            # print("mean_fail")
            # side_x = tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)*(1-match))
            # print("side_x",side_x)
            # side_y = (tf.reduce_sum(tf.abs(1-match))+1e-20)
            # print("side_y",side_y)
            # div = tf.reduce_sum(side_x/side_y)
            # print("div",div)
            mean_fail = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(evidence,1, keepdims=True)*(1-match)) / (tf.reduce_sum(tf.abs(1-match))+1e-20) )
            # print('end')
            self.add_metric(normal_loss, name=self.name+"normal_loss")
            self.add_metric(evidence, name=self.name+"_evidence")
            self.add_metric(mean_avg, name=self.name+"_mean_ev_avg")
            self.add_metric(mean_succ, name=self.name+"_mean_ev_succ")
            self.add_metric(mean_fail, name=self.name+"_mean_ev_fail")
            # print('metrics')
            self.add_loss(total_loss)
            # print("pred",inputs)
            # print("loss")
            #NOTE
            # The total loss is different from parts_loss because it includes the regularization term.
            # In other words, loss is computed as loss = parts_loss + k*R, where R is the regularization term 
            # (typically the L1 or L2 norm of the model's weights) and k a hyperparameter that controls the 
            # contribution of the regularization loss in the total loss.
            return outputs
    

In [34]:
outputs =[]
inputs = keras.Input(shape=(784,))
targets = keras.Input(shape=(10,),name='targets')
x = layers.Flatten(input_shape=(28,28))(inputs)
x = layers.Dense(512, activation="relu")(x)
x= layers.Dropout(0.2)(x)
#exit 2
x = layers.Dense(512, activation="relu")(x)
x= layers.Dropout(0.2)(x)
#exit 3
x = layers.Dense(512, activation="relu")(x)
x= layers.Dropout(0.2)(x)
#exit 4
x = layers.Dense(512, activation="relu")(x)
x= layers.Dropout(0.2)(x)
#exit 5
x = layers.Dense(512, activation="relu")(x)
x= layers.Dropout(0.2)(x)
#exit 1 The main branch exit is refered to as "exit 1" or "main exit" to avoid confusion when adding addtional exits
# output1 = layers.Dense(10, name="output1")(x)
# softmax = layers.Softmax()(output1)
output = EvidenceEndpoint(10, activation='relu')(x,targets)

outputs.append(output)
print(len(outputs))
model = keras.Model(inputs=[inputs,targets], outputs=outputs, name="mnist_model_normal")
model.summary()
#visualize_model(model,"mnist_normal")
print(len(model.outputs))


1
Model: "mnist_model_normal"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 784)          0           input_5[0][0]                    
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 512)          401920      flatten_4[0][0]                  
__________________________________________________________________________________________________
dropout_20 (Dropout)            (None, 512)          0           dense_20[0][0]                   
_______________________________________________________________________________

In [35]:
model.compile( optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'],run_eagerly=True)
model.fit(train_ds,
          epochs=50,
          validation_data=validation_ds,
          validation_freq=2)

Epoch 1/50
187/187 [==============================] - 6s 32ms/step - loss: 0.9917 - accuracy: 0.0981 - evidence_endpoint_3normal_loss: 0.9427 - evidence_endpoint_3_evidence: 0.9803 - evidence_endpoint_3_mean_ev_avg: 9.8026 - evidence_endpoint_3_mean_ev_succ: 9.7892 - evidence_endpoint_3_mean_ev_fail: 9.8039
Epoch 2/50
111/187 [================>.............] - ETA: 2s - loss: 0.9895 - accuracy: 0.1170 - evidence_endpoint_3normal_loss: 0.9413 - evidence_endpoint_3_evidence: 1.0039 - evidence_endpoint_3_mean_ev_avg: 10.0389 - evidence_endpoint_3_mean_ev_succ: 10.0462 - evidence_endpoint_3_mean_ev_fail: 10.0381

KeyboardInterrupt: 